In [1]:
#import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder


from scipy.stats.distributions import gamma
from scipy.stats.distributions import poisson
from scipy.stats.distributions import expon

import statistics
from scipy.stats import norm
from scipy import stats
import scipy.stats

import matplotlib.pyplot as plt



In [2]:
#import data
data = pd.read_csv("../TOKEN BUCKET ALGORITHM/H264-data/silence_of_the_lamb_G16B7_QP10.csv",infer_datetime_format=True,
                  parse_dates=['arrival_time'], index_col='arrival_time', 
                   names=['decoded_frame','arrival_time', 'video_frame', 
                                         'frame_size', 'x', 'y', 'z'])

In [3]:
data.head()

,decoded_frame,video_frame,frame_size,x,y,z
arrival_time,,,,,,
0.00000,0,I,536,98.19,92.17,92.17
266.66667,8,P,152,98.19,92.17,92.17
33.33333,1,B,136,98.19,92.17,92.17
66.66667,2,B,136,98.19,92.17,92.17
100.00000,3,B,136,98.19,92.17,92.17


In [4]:
#drop decoded frame column
data.drop(('decoded_frame'), axis=1, inplace=True)

data.drop(('x'), axis=1, inplace=True)
data.drop(('y'), axis=1, inplace=True)
data.drop(('z'), axis=1, inplace=True)

In [5]:
# Label encoding of species column numerically
le = LabelEncoder()
le.fit(data['video_frame'])
print(list(le.classes_))
data['video_frame'] = le.transform(data['video_frame'])

['B', 'I', 'P']


In [6]:
data.head()

,video_frame,frame_size
arrival_time,,
0.00000,1,536
266.66667,2,152
33.33333,0,136
66.66667,0,136
100.00000,0,136


In [7]:
#Drop NA values
data.frame_size.dropna(0,inplace= True)

In [8]:
data.reset_index(inplace=True)

In [9]:
data.head()

,arrival_time,video_frame,frame_size
0,0.00000,1,536
1,266.66667,2,152
2,33.33333,0,136
3,66.66667,0,136
4,100.00000,0,136


In [10]:
#Sort the arrival time 

data.sort_values('arrival_time', inplace=True)
data.reset_index(drop=True, inplace=True)

In [11]:
data.head(20)
#I = 1, P=2, B= 0

,arrival_time,video_frame,frame_size
0,0.00000,1,536
1,33.33333,0,136
2,66.66667,0,136
3,100.00000,0,136
4,133.33333,0,136
5,166.66667,0,136
6,200.00000,0,136
7,233.33333,0,136
8,266.66667,2,152
9,300.00000,0,136


In [12]:
#Check for missing values

data.isnull().sum()

arrival_time    0
video_frame     0
frame_size      0
dtype: int64

***Define the parameters***

In [13]:
peak_frame_i = max(data[data["video_frame"]==1].frame_size)
peak_frame_b = max(data[data["video_frame"]==0].frame_size)
peak_frame_p = max(data[data["video_frame"]==2].frame_size)

In [14]:
mean_frame_i = np.ceil(np.mean(data[data["video_frame"]== 1].frame_size))
mean_frame_b = np.ceil(np.mean(data[data["video_frame"]== 0].frame_size))
mean_frame_p = np.ceil(np.mean(data[data["video_frame"]== 2].frame_size))

In [15]:
print ('peak_frame i = {}, peak_frame b = {}, peak_frame p = {}'.format( peak_frame_i,peak_frame_b,peak_frame_p))
print ('\nmean_frame i = {}, mean_frame b = {}, mean_frame p = {}'.format(mean_frame_i,mean_frame_b,mean_frame_p))

peak_frame i = 668960, peak_frame b = 393200, peak_frame p = 533048

mean_frame i = 189430.0, mean_frame b = 47503.0, mean_frame p = 132013.0


In [16]:
len(data.frame_size)

53993

In [17]:
peak_frame = max(data.frame_size)
print ('peak_frame', peak_frame)

mean_frame = np.mean(data.frame_size)
mean_frame = np.ceil(mean_frame)
print('mean_frame', mean_frame)

num_of_frames = data.shape[0]
total = np.sum(data.frame_size)
print(num_of_frames)
print(total)

peak_frame 668960
mean_frame 61657.0
53993
3329032160


***Problem statement***

In [18]:

#Calculate the following token generation rate for each frame type (I, P and B):

#I-frame (token generation of I-frame)
I = peak_frame_i/ mean_frame_i
I= np.ceil(I)
print('I rate', I)

# P-frame (token generation rate of P-frame)
P = I * mean_frame_p
P = np.ceil(P)
print ('P rate', P)

# B-frame (token generation rate of B-frame)
B = I * mean_frame_b
B = np.ceil(B)
print('B rate', B)

I rate 4.0
P rate 528052.0
B rate 190012.0


In [19]:
def token_generation_rate(video_frame):
    if video_frame == 0: #B= 0
        token_generation_rate = mean_frame_b
    elif video_frame == 1: #I =1
        token_generation_rate = mean_frame_i
    else: #P=2
        token_generation_rate = mean_frame_p


In [20]:
#Token Bucket algorithm using mean token generation rate

for m in range(10):  # 
    byte_sent = 0   # incoming packet forwarded 
    deleted_bytes = 0
    bucket_size = 0  #  token bucket_size
    marked_bytes = 0  # non conforming packets marked
    j = 0
    l = 0
    max_bucket_size = m * peak_frame  # maximum bucket size - we increased the bucket size to 2,3,4.. 
                                       
    
    for i in range(data.shape[0]):
        video_frame = data.loc[i,"video_frame"]
        
        if video_frame == 0: #B= 0
            token_generation_rate = B
        elif video_frame == 1: #I =1
            token_generation_rate = I
        else: #P=2
            token_generation_rate = P
        
        #video_frame = data.loc[i,"video_frame"]
        #frame_type = data["video_frame"].iloc[i]
        #token_rate = token_generation_rate(frame_type)
        
        
        bucket_size += token_generation_rate  # Finally assign the above solution HERE (token generation rate)
            
            
        if bucket_size > max_bucket_size: 
            bucket_size = max_bucket_size 
            #l += 1
        
        if data.loc[i, 'frame_size'] <= bucket_size:  
                                                        
            byte_sent += data.loc[i, 'frame_size']
            bucket_size = (bucket_size - data.loc[i, 'frame_size'])  
                                                                      
            
        else:
            byte_sent += bucket_size 
            marked_bytes += (data.loc[i, 'frame_size'] - bucket_size)  
            bucket_size = 0  
            
    percent = marked_bytes * 100 / total
    print(percent)
            


100.0
1.8784663227765273
1.692951383203219
1.6125724661067857
1.5486863905814596
1.4961154355444857
1.4559259769962691
1.4157365184480524
1.3755470598998358
1.3353576013516193
